In [1]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import skeletondef as skd
from torch.utils.data import TensorDataset, DataLoader, random_split
from tqdm import tqdm

from PFNNHiddenLayer import PFNNHiddenLayer
from PFNNBiasLayer import PFNNBiasLayer

torch.manual_seed(42)
np.random.seed(42)

In [2]:
# ensure GPU is available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}")

In [3]:
# load data
X = np.float32(np.loadtxt('./data/Input.txt'))
Y = np.float32(np.loadtxt('./data/Output.txt'))
P = np.float32(np.loadtxt('./data/Phases.txt'))
print(X.shape, Y.shape, P.shape)

(124610, 342) (124610, 311) (124610,)


In [4]:
# calculate mean and std
Xmean, Xstd = X.mean(axis=0), X.std(axis=0)
Ymean, Ystd = Y.mean(axis=0), Y.std(axis=0)

j = skd.JOINT_NUM
w = ((60*2)//10)

Xstd[w*0:w* 1] = Xstd[w*0:w* 1].mean() # Trajectory Past Positions
Xstd[w*1:w* 2] = Xstd[w*1:w* 2].mean() # Trajectory Future Positions
Xstd[w*2:w* 3] = Xstd[w*2:w* 3].mean() # Trajectory Past Directions
Xstd[w*3:w* 4] = Xstd[w*3:w* 4].mean() # Trajectory Future Directions
Xstd[w*4:w*10] = Xstd[w*4:w*10].mean() # Trajectory Gait

# mask out unused joints in input
joint_weights = np.array(skd.JOINT_WEIGHTS).repeat(3)

Xstd[w*10+j*3*0:w*10+j*3*1] = Xstd[w*10+j*3*0:w*10+j*3*1].mean() / (joint_weights * 0.1) # Pos
Xstd[w*10+j*3*1:w*10+j*3*2] = Xstd[w*10+j*3*1:w*10+j*3*2].mean() / (joint_weights * 0.1) # Vel
Xstd[w*10+j*3*2:          ] = Xstd[w*10+j*3*2:          ].mean() # Terrain

Ystd[0:2] = Ystd[0:2].mean() # Translational Velocity
Ystd[2:3] = Ystd[2:3].mean() # Rotational Velocity
Ystd[3:4] = Ystd[3:4].mean() # Change in Phase
Ystd[4:8] = Ystd[4:8].mean() # Contacts

Ystd[8+w*0:8+w*1] = Ystd[8+w*0:8+w*1].mean() # Trajectory Future Positions
Ystd[8+w*1:8+w*2] = Ystd[8+w*1:8+w*2].mean() # Trajectory Future Directions

Ystd[8+w*2+j*3*0:8+w*2+j*3*1] = Ystd[8+w*2+j*3*0:8+w*2+j*3*1].mean() # Pos
Ystd[8+w*2+j*3*1:8+w*2+j*3*2] = Ystd[8+w*2+j*3*1:8+w*2+j*3*2].mean() # Vel
Ystd[8+w*2+j*3*2:8+w*2+j*3*3] = Ystd[8+w*2+j*3*2:8+w*2+j*3*3].mean() # Rot

# save mean / std / min / max

Xmean.astype(np.float32).tofile('./weights/Xmean.bin')
Ymean.astype(np.float32).tofile('./weights/Ymean.bin')
Xstd.astype(np.float32).tofile('./weights/Xstd.bin')
Ystd.astype(np.float32).tofile('./weights/Ystd.bin')

# normalize data
X = (X - Xmean) / Xstd
Y = (Y - Ymean) / Ystd

In [5]:
class PhaseFunctionedNetwork(nn.Module):
    def __init__(self, input_shape=1, output_shape=1, dropout=0.7):
        super(PhaseFunctionedNetwork, self).__init__()

        self.nslices = 4
        self.dropout0 = nn.Dropout(p=dropout)
        self.W0 = PFNNHiddenLayer((self.nslices, 512, input_shape-1))
        self.activation0 = nn.ELU()
        
        self.dropout1 = nn.Dropout(p=dropout)
        self.W1 = PFNNHiddenLayer((self.nslices, 512, 512))
        self.activation1 = nn.ELU()
        
        self.dropout2 = nn.Dropout(p=dropout)
        self.W2 = PFNNHiddenLayer((self.nslices, output_shape, 512))

        self.layers = nn.ModuleList([self.W0, self.W1, self.W2])

    def forward(self, input):
        pscale = self.nslices * input[:, -1]
        pamount = pscale % 1.0

        pindex_1 = torch.floor(pscale).long() % self.nslices
        pindex_0 = (pindex_1 - 1) % self.nslices
        pindex_2 = (pindex_1 + 1) % self.nslices
        pindex_3 = (pindex_1 + 2) % self.nslices

        Wamount = pamount.unsqueeze(1).unsqueeze(1)
        bamount = pamount.unsqueeze(1)

        def cubic(y0, y1, y2, y3, mu):
            return (
                (-0.5 * y0 + 1.5 * y1 - 1.5 * y2 + 0.5 * y3) * mu ** 3 +
                (y0 - 2.5 * y1 + 2.0 * y2 - 0.5 * y3) * mu ** 2 +
                (-0.5 * y0 + 0.5 * y2) * mu +
                y1
            )

        self.W0.W.data = cubic(self.W0.W[pindex_0], self.W0.W[pindex_1], self.W0.W[pindex_2], self.W0.W[pindex_3], Wamount)
        self.W0.b.data = cubic(self.W0.b[pindex_0], self.W0.b[pindex_1], self.W0.b[pindex_2], self.W0.b[pindex_3], bamount)

        self.W1.W.data = cubic(self.W1.W[pindex_0], self.W1.W[pindex_1], self.W1.W[pindex_2], self.W1.W[pindex_3], Wamount)
        self.W1.b.data = cubic(self.W1.b[pindex_0], self.W1.b[pindex_1], self.W1.b[pindex_2], self.W1.b[pindex_3], bamount)
        
        self.W2.W.data = cubic(self.W2.W[pindex_0], self.W2.W[pindex_1], self.W2.W[pindex_2], self.W2.W[pindex_3], Wamount)
        self.W2.b.data = cubic(self.W2.b[pindex_0], self.W2.b[pindex_1], self.W2.b[pindex_2], self.W2.b[pindex_3], bamount)

        # input layer
        x = input[:, :-1]
        x = self.dropout0(x)
        # first hidden layer
        x = self.W0(x.unsqueeze(1)).squeeze(1)
        x = self.activation0(x)
        x = self.dropout1(x)
        # second hidden layer
        x = self.W1(x.unsqueeze(1)).squeeze(1)
        x = self.activation1(x)
        x = self.dropout2(x)
        # output layer
        x = self.W2(x.unsqueeze(1)).squeeze(1)

        return x
    
    def cost(self):
        costs = 0
        for layer in self.layers:
            if hasattr(layer, 'cost'):
                costs += layer.cost()
        return costs / len(self.layers)

In [6]:
# try importing data directly
# X = np.float32(np.loadtxt('./X.txt'))
# Y = np.float32(np.loadtxt('./Y.txt'))
# P = np.float32(np.loadtxt('./P.txt'))

In [18]:
# append phase as additional feature
input = torch.tensor(np.concatenate([X, P [..., np.newaxis]], axis=-1))
target = torch.tensor(Y)

dataset = TensorDataset(input[:100000], target[:100000])

BATCH_SIZE = 32
train_dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False)

In [19]:
# ensure GPU is available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}")
device = "cuda"

# custom loss function
def loss_func(output, target, model):
    loss = torch.mean((output - target)**2) + model.cost()
    return loss

In [21]:
model = PhaseFunctionedNetwork(input_shape=input.shape[1], output_shape=311)
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=0.0001)
epochs=2

for epoch in range(epochs):
    model.train()
    loss_list = []
    for i, batch in enumerate(tqdm(train_dataloader)):
        input, target = batch
        input, target = input.to(device), target.to(device)

        # forward pass
        output = model(input)
        loss = loss_func(output, target, model)
        loss_list.append(loss.item())

        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {np.average(loss_list)}')


  0%|          | 0/100 [00:02<?, ?it/s]


KeyboardInterrupt: 

In [13]:
# precompute weights
for i in range(50):
    
    pscale = model.nslices*(float(i)/50)
    pamount = pscale % 1.0
    
    pindex_1 = int(pscale) % model.nslices
    pindex_0 = (pindex_1-1) % model.nslices
    pindex_2 = (pindex_1+1) % model.nslices
    pindex_3 = (pindex_1+2) % model.nslices
    
    def cubic(y0, y1, y2, y3, mu):
        return (
            (-0.5*y0+1.5*y1-1.5*y2+0.5*y3)*mu*mu*mu + 
            (y0-2.5*y1+2.0*y2-0.5*y3)*mu*mu + 
            (-0.5*y0+0.5*y2)*mu +
            (y1))
    
    W0 = cubic(model.W0.W[pindex_0], model.W0.W[pindex_1], model.W0.W[pindex_2], model.W0.W[pindex_3], pamount).cpu().detach().numpy()
    W1 = cubic(model.W1.W[pindex_0], model.W1.W[pindex_1], model.W1.W[pindex_2], model.W1.W[pindex_3], pamount).cpu().detach().numpy()
    W2 = cubic(model.W2.W[pindex_0], model.W2.W[pindex_1], model.W2.W[pindex_2], model.W2.W[pindex_3], pamount).cpu().detach().numpy()

    b0 = cubic(model.W0.b[pindex_0], model.W0.b[pindex_1], model.W0.b[pindex_2], model.W0.b[pindex_3], pamount).cpu().detach().numpy()
    b1 = cubic(model.W1.b[pindex_0], model.W1.b[pindex_1], model.W1.b[pindex_2], model.W1.b[pindex_3], pamount).cpu().detach().numpy()
    b2 = cubic(model.W2.b[pindex_0], model.W2.b[pindex_1], model.W2.b[pindex_2], model.W2.b[pindex_3], pamount).cpu().detach().numpy()
    
    W0.astype(np.float32).tofile('./weights/W0_%03i.bin' % i)
    W1.astype(np.float32).tofile('./weights/W1_%03i.bin' % i)
    W2.astype(np.float32).tofile('./weights/W2_%03i.bin' % i)
    
    b0.astype(np.float32).tofile('./weights/b0_%03i.bin' % i)
    b1.astype(np.float32).tofile('./weights/b1_%03i.bin' % i)
    b2.astype(np.float32).tofile('./weights/b2_%03i.bin' % i)